In [22]:
from jinja2 import Environment, FileSystemLoader
import pandas as pd
from weasyprint import HTML ,CSS
import os
import zipfile
import glob

df = pd.read_csv('datafile.csv')

env = Environment(loader=FileSystemLoader('templates'))
template = env.get_template('report.html')

for i,row in df.iterrows():
    filename = 'report'+str(row['CF Reference ID'])
    if(row['GST number']!=''):
        ogst = row['GST number']
    html = template.render(date=str(row['Date']),
                            invoiceNumber = str(row['CF Reference ID']),
                            providerName=str(row['Load owner name']),
                            providerCode= str(row['GST']),
                            providerLocation = str(row['Source City']),
                            ownerName = str(row['Lorry owner name']),
                            ownerGst = ogst,
                            ownerLocation = str(row['Destination City']),
                            price = str(row['Date']),
                            product = str(row['Material ']),
                            quantity = str(row['Weight (Ton)']),
                            amount1 = str(row['Total Amount']) ,
                            amount2 = str(row['Amount paid by loader']) ,
                            amout3 = str(row['Discount Given (INR)']) ,
                            paymentType = str(row['Payment Time']),
                            utr = str(row['UTR Number']))
    with open('report.html','w') as f:
        f.write(html)
    css = CSS(string = """
    
			.invoice-box {
				max-width: 800px;
				margin: auto;
				padding: 30px;
				border: 1px solid #eee;
				box-shadow: 0 0 10px rgba(0, 0, 0, 0.15);
				font-size: 16px;
				line-height: 24px;
				font-family: 'Helvetica Neue', 'Helvetica', Helvetica, Arial, sans-serif;
				color: #555;
			}
            .text-center{text-align:center;}

			.invoice-box table {
				width: 100%;
				line-height: inherit;
				text-align: left;
			}

			.invoice-box table td {
				padding: 5px;
				vertical-align: top;
			}

			.invoice-box table tr td:nth-child(2) {
				text-align: right;
			}

			.invoice-box table tr.top table td {
				padding-bottom: 20px;
			}

			.invoice-box table tr.top table td.title {
				font-size: 45px;
				line-height: 45px;
				color: #333;
			}

			.invoice-box table tr.information table td {
				padding-bottom: 40px;
			}

			.invoice-box table tr.heading td {
				background: #eee;
				border-bottom: 1px solid #ddd;
				font-weight: bold;
			}

			.invoice-box table tr.details td {
				padding-bottom: 20px;
			}

			.invoice-box table tr.item td {
				border-bottom: 1px solid #eee;
			}

			.invoice-box table tr.item.last td {
				border-bottom: none;
			}

			.invoice-box table tr.total td:nth-child(2) {
				border-top: 2px solid #eee;
				font-weight: bold;
			}

			@media only screen and (max-width: 600px) {
				.invoice-box table tr.top table td {
					width: 100%;
					display: block;
					text-align: center;
				}

				.invoice-box table tr.information table td {
					width: 100%;
					display: block;
					text-align: center;
				}
			}

			/** RTL **/
			.invoice-box.rtl {
				direction: rtl;
				font-family: Tahoma, 'Helvetica Neue', 'Helvetica', Helvetica, Arial, sans-serif;
			}

			.invoice-box.rtl table {
				text-align: right;
			}

			.invoice-box.rtl table tr td:nth-child(2) {
				text-align: left;
			}
    """)
    HTML('report.html').write_pdf('reports/'+filename+'.pdf',stylesheets=[css])
    
def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file), 
                       os.path.relpath(os.path.join(root, file), 
                                       os.path.join(path, '..')))

with zipfile.ZipFile('invoices.zip', 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipdir('reports/', zipf)

pdffiles = glob.glob('reports/*.pdf')
for f in pdffiles:
    try:
        os.remove(f)
    except OSError as e:
        print("Error: %s : %s" % (f, e.strerror))